In [1]:
from gibbs.dataclass import GibbsResult, get_results
import matplotlib.pyplot as plt
import numpy as np
from gibbs.learning.bayesian_learning import BayesianLearning
from gibbs.learning.constraint_matrix import ConstraintMatrixFactory, DumbConstraintMatrixFactory
from qiskit.quantum_info import Statevector
from gibbs.learning.klocal_pauli_basis import KLocalPauliBasis
from scipy.linalg import block_diag
from scipy.sparse import bmat
from qiskit.quantum_info import state_fidelity
from gibbs.utils import number_of_elements, simple_purify_hamiltonian, spectral_dec,printarray
import plotly_express as px
# plt.rcParams['text.usetex'] = True

%load_ext autoreload
%autoreload 2

gibbsresults = get_results("../saved_simulations/turbo/random1localheisenberg")

In [2]:
n=4
k=2
nfields = 20
shots = 1e6
prep_noise = 1e-1
basisH = KLocalPauliBasis(k,n,False)
cmat = ConstraintMatrixFactory(n,k,k+1)
c_original = np.zeros(basisH.size)
c_original[basisH.pauli_to_num("XX"+"I"*(n-2)):basisH.pauli_to_num("I"*(n-2)+"XX")+1]=1
c_original_prior = c_original.copy()
# c_original[basisH.pauli_to_num("XX"+"I"*(n-2)):] += np.random.normal(0,prep_noise,size=c_original[basisH.pauli_to_num("XX"+"I"*(n-2)):].size)
c_original += np.random.normal(0,prep_noise,size=c_original.size)

print(f"We will need to sample {KLocalPauliBasis(2*k,n,False).size} Paulis for each one of the states. In total,{int((nfields+1)*shots*KLocalPauliBasis(2*k,n,False).size):.1E}")

We will need to sample 255 Paulis for each one of the states. In total,5.4E+09


In [7]:
timestep =-1
states = [g.state_ansatz(timestep) for g in gibbsresults]
control_fields = [g.coriginal-gibbsresults[0].coriginal for g in gibbsresults]
initial_arguments = {
    "states":states,
    "control_fields": control_fields,
    "cmat_factory": ConstraintMatrixFactory(num_qubits:=gibbsresults[0].num_qubits,3,3),
    "prior_mean": c_original_prior,
    "prior_covariance": (prep_noise,1e-3),
    "sampling_std": 1/np.sqrt(shots),
    "shots": shots
    }
bl = BayesianLearning(**initial_arguments)

In [8]:
regular_cmat_factory=ConstraintMatrixFactory(num_qubits,2,3)
A_regular = regular_cmat_factory.create_cmat(states[0],shots)
print(f"In total we are performing {(regular_shots:=regular_cmat_factory.counter_shots):.3e} shots")
s,v = spectral_dec(A_regular)
candidate = v[-1].copy(); candidate /= np.linalg.norm(candidate)
if np.linalg.norm(c_original/np.linalg.norm(c_original) + candidate) < np.linalg.norm(c_original/np.linalg.norm(c_original) - candidate):
    candidate = -candidate
fig,ax = plt.subplots(1,2,figsize=(10,5))
ax[0].stairs(c_original/np.linalg.norm(c_original),lw=3)
ax[0].stairs(candidate)
ax[1].plot(np.log(s),marker=".")
print(f"We start with a hamiltonian error of:{np.linalg.norm(c_original_prior-c_original)} and end up with {np.linalg.norm(candidate*np.linalg.norm(c_original)-c_original)} ")
print(f"The prior fidelity is: {state_fidelity(simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original_prior)),simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original)))} and the posterior fidelity is: {state_fidelity(simple_purify_hamiltonian(basisH.vector_to_pauli_op(candidate*np.linalg.norm(c_original))),simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original)))}")

In total we are performing 0.000e+00 shots


ValueError: operands could not be broadcast together with shapes (39,) (63,) 

In [ ]:
dumb_cmat_factory=DumbConstraintMatrixFactory(num_qubits,2,3)
A_dumb = dumb_cmat_factory.create_cmat(states[0],shots//6.894)
print(f"In total we are performing {(dumb_shots:=dumb_cmat_factory.counter_shots):.3e} shots")
print(f"That is {dumb_shots/regular_shots:.3e} more times than we needed.")
s,v = spectral_dec(A_dumb)
candidate = v[-1].copy(); candidate /= np.linalg.norm(candidate)
if np.linalg.norm(c_original/np.linalg.norm(c_original) + candidate) < np.linalg.norm(c_original/np.linalg.norm(c_original) - candidate):
    candidate = -candidate
fig,ax = plt.subplots(1,2,figsize=(10,5))
ax[0].stairs(c_original/np.linalg.norm(c_original),lw=3)
ax[0].stairs(candidate)
ax[1].plot(np.log(s),marker=".")
print(f"We start with a hamiltonian error of:{np.linalg.norm(c_original_prior-c_original)} and end up with {np.linalg.norm(candidate*np.linalg.norm(c_original)-c_original)} ")
print(f"The prior fidelity is: {state_fidelity(simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original_prior)),simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original)))} and the posterior fidelity is: {state_fidelity(simple_purify_hamiltonian(basisH.vector_to_pauli_op(candidate*np.linalg.norm(c_original))),simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original)))}")

In [9]:
A = bl.block_control_matrix(range(5))
s,v = spectral_dec(A)
candidate = v[-1][:c_original.size].copy(); candidate /= np.linalg.norm(candidate)
if np.linalg.norm(c_original/np.linalg.norm(c_original) + candidate) < np.linalg.norm(c_original/np.linalg.norm(c_original) - candidate):
    candidate = -candidate  
fig,ax = plt.subplots(1,2,figsize=(10,5))
ax[0].stairs(c_original/np.linalg.norm(c_original),lw=3)
ax[0].stairs(candidate)
ax[1].plot(np.log(s),marker=".")
print(f"We start with a hamiltonian error of:{np.linalg.norm(c_original_prior-c_original)} and end up with {np.linalg.norm(candidate*np.linalg.norm(c_original)-c_original)} ")
print(f"The prior fidelity is: {state_fidelity(simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original_prior)),simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original)))} and the posterior fidelity is: {state_fidelity(simple_purify_hamiltonian(basisH.vector_to_pauli_op(candidate*np.linalg.norm(c_original))),simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original)))}")

KeyboardInterrupt: 

In [10]:
means = [bl.current_mean]
covs = [bl.total_cov]
for indexes in ([(0,1,2,3),(0,4,5,6),(0,7,8,9),(0,10,11,12),(0,13,14,15)]):
    posterior_mean = bl.update_mean(indexes,options={"maxiter": 100, "xrtol": 1e-3  , "disp": True})
    posterior_cov = bl.update_cov(posterior_mean,indexes)
    means.append(bl.current_mean)
    covs.append(bl.total_cov)

4 (828, 828) (660,) (39,)


AssertionError: Size of x0: 660 and constraint matrix: (828, 828) don't match

In [ ]:
px.imshow(covs[-1])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,5))
ax[0].stairs(c_original,label="preparation")
ax[0].stairs(c_original_prior,label = "prior")
ax[0].stairs(posterior_mean[:c_original.size],label="posterior")
ax[0].legend()
width = 0.9
ax[1].bar(np.arange(bl.size),np.abs(c_original-c_original_prior),width,label="prior error",lw=2,fill=True)
ax[1].bar(np.arange(bl.size),np.abs(c_original-posterior_mean[:c_original.size]),0.6*width,label="posterior error",fill=True)
ax[1].stairs(posterior_cov.diagonal()[:c_original.size],np.arange(bl.size+1)-1/2,label="std",color="red")
ax[1].legend()
# ax[2].stairs(posterior_mean,label="posterior")
# ax[2].stairs(x_ideal,label="ideal")
# ax[2].stairs(posterior_cov.diagonal(),label="posterior std")
# ax[2].legend()
print(f"We start with a hamiltonian error of:{np.linalg.norm(c_original_prior-c_original)} and end up with {np.linalg.norm(posterior_mean[:c_original.size]-c_original)} ")
print(f"The prior fidelity is: {state_fidelity(simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original_prior)),simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original)))} and the posterior fidelity is: {state_fidelity(simple_purify_hamiltonian(basisH.vector_to_pauli_op(posterior_mean[:c_original.size])),simple_purify_hamiltonian(basisH.vector_to_pauli_op(c_original)))}")